# Análise da influência das bases de dados nas queries expandidas pelo AQE

Nesta análise, vamos identificar a influência das bases de dados nas expansões realizadas pelo AQE, além de quais os pesos mais adequados para os fatores de boosting de consulta ao elasticsearch.

Nesta análise vamos utilizar as queries da [base REGIS](https://github.com/Petroles/regis-collection) para medir a performance do ranking, como também as 12 bases de dados utilizadas na expansão da consulta do AQE:

* 01_DicionarioPetroleo_Curado_ComSinonimos
* 02_Tesauro_comTraducoesRegis
* 03_ListaCurada
* 05_InstanciasBDIEP_Ativo_Bloco_Campo
* 06_Termos_TabelaPocosANP2019
* 07_Pocos_TabelaPocosANP2019
* 08_Pocos_BDIEP_com2ou3_Siglas
* 09_Glossario_ANP
* 10_List_of_abbreviations_curada
* 11_Lista_Feita_a_Mao
* 12_Partex_Acronymis_Oil_Gas
* 13_Lista_MWE

## Carregando libs

In [ ]:
import json
from pathlib import Path
import yaml
import pandas as pd
import numpy as np
import plotly.express as px

from utils.utils import adjust_new_expanded_queries, create_metrics, create_new_metrics,\
    create_validation_dataset, get_databases_queries, create_new_validation_dataset, \
    get_expanded_queries, make_elasticsearch_queries, make_elasticsearch_new_queries

## Carregando as configurações e bases dados

In [ ]:
with open("../conf/config.yaml", "r") as yamlfile:
    cfg = yaml.safe_load(yamlfile)

In [ ]:
data_path = Path("../../dados/vocabulario_oil_gas")
databases = dict()

In [ ]:
with open(data_path.joinpath("01_DicionarioPetroleo_Curado_ComSinonimos.csv"), "r", encoding='unicode_escape') as f:
    databases["01_DicionarioPetroleo_Curado_ComSinonimos"] = f.read().replace(";", "\n").split("\n")
    databases["01_DicionarioPetroleo_Curado_ComSinonimos"] = [e.strip().lower() for e in databases["01_DicionarioPetroleo_Curado_ComSinonimos"]]
print(databases["01_DicionarioPetroleo_Curado_ComSinonimos"][:20])

In [ ]:
with open(data_path.joinpath("02_Tesauro_comTraducoesRegis.csv"), "r", encoding='unicode_escape') as f:
    databases["02_Tesauro_comTraducoesRegis"] = f.read().replace("#", ";").replace("\t", "").replace(";", "\n").split("\n")
    databases["02_Tesauro_comTraducoesRegis"] = [e.strip().lower() for e in databases["02_Tesauro_comTraducoesRegis"] if e != ""]
print(databases["02_Tesauro_comTraducoesRegis"][:20])

In [ ]:
with open(data_path.joinpath("03_ListaCurada.csv"), "r", encoding='unicode_escape') as f:
    databases["03_ListaCurada"] = f.read().split("\n")
    databases["03_ListaCurada"] = [e.strip().lower() for e in databases["03_ListaCurada"] if e != ""]
print(databases["03_ListaCurada"][:20])

In [ ]:
with open(data_path.joinpath("05_InstanciasBDIEP_Ativo_Bloco_Campo.csv"), "r", encoding='unicode_escape') as f:
    databases["05_InstanciasBDIEP_Ativo_Bloco_Campo"] = f.read().replace(";", "\n").split("\n")
    databases["05_InstanciasBDIEP_Ativo_Bloco_Campo"] = [e.strip().lower() for e in databases["05_InstanciasBDIEP_Ativo_Bloco_Campo"] if e != ""]
print(databases["05_InstanciasBDIEP_Ativo_Bloco_Campo"][:20])

In [ ]:
with open(data_path.joinpath("06_Termos_TabelaPocosANP2019.csv"), "r", encoding='unicode_escape') as f:
    databases["06_Termos_TabelaPocosANP2019"] = f.read().replace(";", "\n").split("\n")
    databases["06_Termos_TabelaPocosANP2019"] = [e.strip().lower() for e in databases["06_Termos_TabelaPocosANP2019"] if e != ""]
print(databases["06_Termos_TabelaPocosANP2019"][:20])

In [ ]:
with open(data_path.joinpath("07_Pocos_TabelaPocosANP2019.csv"), "r", encoding='unicode_escape') as f:
    databases["07_Pocos_TabelaPocosANP2019"] = f.read().replace(";", "\n").split("\n")
    databases["07_Pocos_TabelaPocosANP2019"] = [e.strip().lower() for e in databases["07_Pocos_TabelaPocosANP2019"] if e != ""]
print(databases["07_Pocos_TabelaPocosANP2019"][:20])

In [ ]:
with open(data_path.joinpath("08_Pocos_BDIEP_com2ou3_Siglas.csv"), "r", encoding='utf-8-sig') as f:
    databases["08_Pocos_BDIEP_com2ou3_Siglas"] = f.read().replace(";", "\n").split("\n")
    databases["08_Pocos_BDIEP_com2ou3_Siglas"] = [e.strip().lower() for e in databases["08_Pocos_BDIEP_com2ou3_Siglas"] if e != ""]
print(databases["08_Pocos_BDIEP_com2ou3_Siglas"][:20])

In [ ]:
with open(data_path.joinpath("09_Glossario_ANP.csv"), "r", encoding='unicode_escape') as f:
    databases["09_Glossario_ANP"] = f.read().replace(";", "\n").split("\n")
    databases["09_Glossario_ANP"] = [e.strip().lower() for e in databases["09_Glossario_ANP"] if e != ""]
print(databases["09_Glossario_ANP"][:20])

In [ ]:
with open(data_path.joinpath("10_List_of_abbreviations_curada.csv"), "r", encoding='unicode_escape') as f:
    databases["10_List_of_abbreviations_curada"] = f.read().replace(";", "\n").split("\n")
    databases["10_List_of_abbreviations_curada"] = [e.strip().lower() for e in databases["10_List_of_abbreviations_curada"] if e != ""]
print(databases["10_List_of_abbreviations_curada"][:20])

In [ ]:
with open(data_path.joinpath("11_Lista_Feita_a_Mao.csv"), "r", encoding='unicode_escape') as f:
    databases["11_Lista_Feita_a_Mao"] = f.read().replace(",", ";").replace(";", "\n").split("\n")
    databases["11_Lista_Feita_a_Mao"] = [e.strip().lower() for e in databases["11_Lista_Feita_a_Mao"] if e != ""]
print(databases["11_Lista_Feita_a_Mao"][:20])

In [ ]:
with open(data_path.joinpath("12_Partex_Acronymis_Oil_Gas.csv"), "r") as f:
    databases["12_Partex_Acronymis_Oil_Gas"] = f.read().replace(";", "\n").split("\n")
    databases["12_Partex_Acronymis_Oil_Gas"] = [e.strip().lower() for e in databases["12_Partex_Acronymis_Oil_Gas"] if e != ""]
print(databases["12_Partex_Acronymis_Oil_Gas"][:20])

In [ ]:
with open(data_path.joinpath("13_Lista_MWE.txt"), "r") as f:
    databases["13_Lista_MWE"] = f.read().replace(",", "\n").split("\n")
    databases["13_Lista_MWE"] = [e.strip().lower() for e in databases["13_Lista_MWE"] if e != ""]
print(databases["13_Lista_MWE"][:20])

## Expansão das queries

Agora vamos coletar as expansões das queries da base REGIS.

In [ ]:
with open("../../dados/regis/regis_queries.json", 'r') as regis_file:
    regis_queries = json.load(regis_file)

In [ ]:
regis_queries = get_expanded_queries(regis_queries)
regis_queries[:5]

## Criando queries para cada base de dados

Agora, para cada base de dados, vamos identificar se cada termo expandido está presente ou não naquela base de dados, zerando o fator de boosting quando não estiver. Para os fatores de boosting vamos experimentar entre 0,1 e 1,5, com intervalos de 0,1. Esse fator multiplicará o valor original de boost do AQE.

In [ ]:
databases_queries = get_databases_queries(databases, regis_queries, np.arange(0.1, 1.6, 0.1))

## Realizando consultas no Elasticsearch

Em posse das queries para cada fator e base de dados, agora vamos realizar as consultas ao elastic search e criar o dataset de validação, o qual possui informações do ground truth da base de dados REGIS.

In [ ]:
ranking_result_df = make_elasticsearch_queries(databases_queries, cfg, 50)

In [ ]:
ranking_result_df.head()

In [ ]:
ground_truth = pd.read_csv("../../dados/regis/regis_ground_truth.csv")
ground_truth.head()

In [ ]:
validation_dataset = create_validation_dataset(ranking_result_df, ground_truth)
validation_dataset.head()

In [ ]:
validation_dataset.to_csv("../data/databases_es_documents.csv", index=False)

In [ ]:
# validation_dataset = pd.read_csv("../data/databases_es_documents.csv")
# validation_dataset.head()

## Análise das consultas no Elasticsearch

Agora vamos criar as métricas para cada base de dados e fator e visualizar os resultados.

### Criando métricas

In [ ]:
metrics_df = create_metrics(validation_dataset)
metrics_df.head()

In [ ]:
metrics_df.to_csv("../data/databases_metrics.csv", index=False)

### Avaliando métricas

Vamos agora avaliar as métricas. Vamos utilizar as seguintes métricas:

* ndcg - Normalized Discounted Cumulative Gain
* map - Mean Average Precision
* eval_prop - Proporção de documentos avaliados

In [ ]:
# metrics_df = pd.read_csv("../data/databases_metrics.csv")
# metrics_df.head()

In [ ]:
data_viz = metrics_df.groupby(
    ["database_name", "factor"]
).agg(
    "mean"
).reset_index(
)

data_viz.head()

In [ ]:
fig = px.line(
    data_viz,
    x="factor",
    y="ndcg",
    color="database_name",
    markers=True,
    title="Performances das bases de dados com diferentes fatores"
).add_hline(
    y=0.7351,
    line_dash="dash",
    line_color="black"
)
fig.show()

É possível ver que todas as bases de dados, a partir de um fator de 0,2 existe uma tendência decrescente do NDCG ao passo que aumenta o fator de multiplicação do boosting.

### Identificando melhor fator de multipliação do boost para cada base de dados

Vamos repetir o processo acima, mas agora com objetivo de identificar qual o melhor fator de multiplicação do boost para cada base de dados, utilizando fatores entre 0 e 0,2.

In [ ]:
databases_queries = get_databases_queries(databases, regis_queries, np.arange(0.01, 0.25, 0.01))

In [ ]:
ranking_result_df = make_elasticsearch_queries(databases_queries, cfg, 50)

In [ ]:
validation_dataset = create_validation_dataset(ranking_result_df, ground_truth)

In [ ]:
metrics_df = create_metrics(validation_dataset)
metrics_df.head()

In [ ]:
data_viz = metrics_df.groupby(
    ["database_name", "factor"]
).agg(
    "mean"
).reset_index(
)

fig = px.line(
    data_viz,
    x="factor",
    y="ndcg",
    color="database_name",
    markers=True,
    title="Performances das bases de dados com diferentes fatores"
).add_hline(
    y=0.7351,
    line_dash="dash",
    line_color="black"
)
fig.show()

Podemos identificar que os seguintes fatores são os melhores para cada base:

* 01_DicionarioPetroleo_Curado_ComSinonimos: 0.03
* 02_Tesauro_comTraducoesRegis: 0.11
* 03_ListaCurada: 0.0
* 05_InstanciasBDIEP_Ativo_Bloco_Campo: 0.01
* 06_Termos_TabelaPocosANP2019: 0.1
* 07_Pocos_TabelaPocosANP2019: 0.0
* 08_Pocos_BDIEP_com2ou3_Siglas: 0.0
* 09_Glossario_ANP: 0.14
* 10_List_of_abbreviations_curada: 0.13
* 11_Lista_Feita_a_Mao: 0.0
* 12_Partex_Acronymis_Oil_Gas: 0.12
* 13_Lista_MWE: 0.08

Vejamos o impacto de cada base de dados no ndcg:

In [ ]:
data_viz.groupby("database_name").agg({"ndcg": "max"}).reset_index().sort_values("ndcg", ascending=False)

### Criando novas queries baseados nos fatores encontrados

Agora vamos ajustar as queries para utilizar os melhores fatores para cada base de dados.

In [ ]:
databases_factors = {
    "01_DicionarioPetroleo_Curado_ComSinonimos": 0.03,
    "02_Tesauro_comTraducoesRegis": 0.11,
    "03_ListaCurada": 0.0,
    "05_InstanciasBDIEP_Ativo_Bloco_Campo": 0.01,
    "06_Termos_TabelaPocosANP2019": 0.1,
    "07_Pocos_TabelaPocosANP2019": 0.0,
    "08_Pocos_BDIEP_com2ou3_Siglas": 0.0,
    "09_Glossario_ANP": 0.14,
    "10_List_of_abbreviations_curada": 0.13,
    "11_Lista_Feita_a_Mao": 0.0,
    "12_Partex_Acronymis_Oil_Gas": 0.12,
    "13_Lista_MWE": 0.08,
}

In [ ]:
new_expanded_queries = adjust_new_expanded_queries(
    databases,
    regis_queries,
    databases_factors
)

In [ ]:
ranking_result_df = make_elasticsearch_new_queries(new_expanded_queries, cfg, 50)
ranking_result_df.head()

In [ ]:
validation_dataset = create_new_validation_dataset(ranking_result_df, ground_truth)
validation_dataset.head()

In [ ]:
metrics_df = create_new_metrics(validation_dataset)
metrics_df.head()

In [ ]:
metrics_df["ndcg"].mean()

É possível observar que o NDCG médio aumentou 2,18% quando comparado com ao NDCG médio da consulta sem o AQE.

## Conclusão

Foi possível identificar que algumas bases de dados são mais importantes que outras na expansão da consulta, onde algumas trazem resultados inferiores comparando sem o uso do AQE. As bases de dados mais importantes são:

1. 06_Termos_TabelaPocosANP2019	com ndcg de **0.759765**
2. 13_Lista_MWE	com ndcg de **0.751475**
3. 09_Glossario_ANP	com ndcg de **0.739109**
4. 12_Partex_Acronymis_Oil_Gas	com ndcg de **0.737709**
5. 01_DicionarioPetroleo_Curado_ComSinonimos	com ndcg de **0.737612**
6. 10_List_of_abbreviations_curada	com ndcg de **0.736354**
7. 05_InstanciasBDIEP_Ativo_Bloco_Campo	com ndcg de **0.735321**

Utilizando os melhores pesos para cada base de dados o NDCG ficou abaixo do NDCG utilizando apenas a base de dados 06_Termos_TabelaPocosANP2019, mas ficou acima das demais bases, atingindo um NDCG de **0.756874**. Tendo em vista que utilizar sete bases traz uma maior variabilidade de documentos e atingiu um NDCG **0.00289** menor, concluímos que utilizar os fatores de boosting sugeridos traz o melhor resultado encontrado até então. Segue os fatores de boosting para cada base de dados:

* 01_DicionarioPetroleo_Curado_ComSinonimos: 0.03
* 02_Tesauro_comTraducoesRegis: 0.11
* 03_ListaCurada: 0.0
* 05_InstanciasBDIEP_Ativo_Bloco_Campo: 0.01
* 06_Termos_TabelaPocosANP2019: 0.1
* 07_Pocos_TabelaPocosANP2019: 0.0
* 08_Pocos_BDIEP_com2ou3_Siglas: 0.0
* 09_Glossario_ANP: 0.14
* 10_List_of_abbreviations_curada: 0.13
* 11_Lista_Feita_a_Mao: 0.0
* 12_Partex_Acronymis_Oil_Gas: 0.12
* 13_Lista_MWE: 0.08